#### setup colab

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/people'

Mounted at /content/drive/
/content/drive/My Drive/people


#### imports

In [270]:
import pandas as p
from scipy import stats
import numpy as np

from sklearn.preprocessing import LabelEncoder

#### load raw

In [363]:
test=pd.read_csv("test.csv");train=pd.read_csv("train.csv");train_raw=train.copy();test_raw=test.copy()

#### EDA

#### data points modification
job_level, person_level,age, Education_level, year_graduated, achievement_target_1(2&3)
- (may need info on which year is this data taken from)
- info from each columns

penyesuaian nilai angka **age** (current_age), **year_graduated** (years_graduated)

mengubah Dtype year_graduated sekaligus nilai yg tidak sesuai menjadi NaN

In [364]:
train['year_graduated']=pd.to_numeric(train['year_graduated'], errors='coerce', downcast ='integer')
# train['year_graduated'].astype('int32') #cannot convert NA or inf to integer
# keduanya dengan asumsi ini tahun 2019
train['current_age']=2019-train['age']
train['years_graduated']=2019-train['year_graduated']
train.drop(['age','year_graduated','Education_level'],axis=1,inplace=True)

 penghapusan sebagian karakter untuk dijadikan bentuk ordinal: **job_level**, **person_level**, dan , **education_level**

In [365]:
train['job_level']=train['job_level'].str.lstrip('JG0')
train['person_level']=train['person_level'].str.lstrip('PG0')
train['education_level']=train['person_level'].str.lstrip('level_')

train['job_level']=pd.to_numeric(train['job_level'], errors='raise')
train['person_level']=pd.to_numeric(train['person_level'], errors='raise')
train['education_level']=pd.to_numeric(train['education_level'], errors='raise')

**achievement_target_1** & **achievement_target_2** diolah lebih lanjut

In [366]:
targets=['achievement_target_1','achievement_target_2']
for target in targets:
    train.loc[(train[target] == 'achiev_< 50%' )|( train[target] == 'Pencapaian < 50%'), target] = 1
    train.loc[(train[target] == 'achiev_50%-100%' )|( train[target] == 'Pencapaian 50%-100%'), target] = 2
    train.loc[(train[target] == 'achiev_100%-150%' )|( train[target] == 'Pencapaian 100%-150%'), target] = 3
    train.loc[(train[target] == 'achiev_> 1.5' )|( train[target] == 'Pencapaian > 1.5'), target] = 4
#     train.loc[train[target] == 'Tidak diberikan target', target] = float('NaN')

    train[target]=pd.to_numeric(train[target], errors='coerce') #NaN given here
    
train['achievement_target_3'] = train['achievement_target_3'].apply(lambda x: 1 if x == 'reached' else 0)


In [367]:
print('1:',train['achievement_target_1'].unique(),'\n2:', train['achievement_target_2'].unique())
train['achievement_target_1']

1: [ 2.  3.  1.  4. nan] 
2: [ 1.  2.  4.  3. nan]


0        2.0
1        2.0
2        3.0
3        2.0
4        1.0
        ... 
22000    2.0
22001    NaN
22002    2.0
22003    3.0
22004    2.0
Name: achievement_target_1, Length: 22005, dtype: float64

#### outliers management
fiturnya banyak, jadi sayang bgt kalo gara2 1 fitur ada outlier lgsg dibuang
outlier ada di: yg dtypenya int dan float


In [368]:
angka_columns = train.loc[:, (train.dtypes == np.float64)|(train.dtypes == np.int64)].columns
                        
#ubah semua outliers jadi NaN (untuk nanti diubah jadi mean??)                        
for col in angka_columns:
    z =     np.abs((train[col] - train[col].mean())/train[col].std(ddof=0))
    #print(z,col)
    train.loc[z > 3, col] = float('NaN')

In [369]:
print('cek perubahan setelah main outlier',train_raw.info(),train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22005 entries, 0 to 22004
Data columns (total 30 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_level                              22005 non-null  object 
 1   job_duration_in_current_job_level      22005 non-null  float64
 2   person_level                           22005 non-null  object 
 3   job_duration_in_current_person_level   22005 non-null  float64
 4   job_duration_in_current_branch         22005 non-null  float64
 5   Employee_type                          21993 non-null  object 
 6   Employee_status                        22005 non-null  object 
 7   gender                                 22005 non-null  object 
 8   age                                    22005 non-null  int64  
 9   marital_status_maried(Y/N)             22005 non-null  object 
 10  number_of_dependences                  22005 non-null  int64  
 11  nu

save dataset sampe checkpoint ngilangin outlier

In [370]:
train.to_csv('train_setelah_buang_outlier.csv')

#### Dtype modification **binary and 1 hot**
sisa 4 kategorikal yang masi dalam bentuk objek, ubah.

employee type 1hot, employee status binary, gender binary, marital_status_married(Y/N) binary, achievement target 3 binary

In [371]:
def biner(df,col):
    for c in col:
        df[c] = LabelEncoder().fit_transform(df[c])
        
biner(train,['Employee_status','gender','marital_status_maried(Y/N)'])

In [372]:
#1hotting every column with object dtype left
def onehot(df_list,col): #list form
    df_list_1h=[]
    for df in df_list:
#         df_obj=df.select_dtypes('object')
        for c in range(len(col)):
#             label_df = pd.DataFrame(df[col[c]], columns=[col[c]])
            print(c)
            dum_df = pd.get_dummies(df[col[c]], columns=[col[c]], prefix=col[c])
            df=df.join(dum_df)
            df=df.drop(col[c], axis=1)
        df_list_1h.append(df)
        
    return df_list_1h
        
col=train.select_dtypes('object').columns
train,=onehot([train],col)

0


#### NaNs management


#### saving processed dataset

#### test

In [373]:
a=[2,3,4,5]
q=[1]

In [374]:
b,c,d,e=a

In [375]:
z,=q
zl=q

In [376]:
z,zl

(1, [1])